# Beautiful Soup


In [ ]:
import urllib2
import csv
from bs4 import BeautifulSoup


In [ ]:
#open url
data_source = 'https://finance.yahoo.com/'
req = urllib2.Request(data_source, headers={'User-Agent': 'Mozilla/5.0'})
page = urllib2.urlopen(req).read()
soup = BeautifulSoup(page, "html5lib")


## Print html data

In [ ]:
print type(soup)
print soup.prettify()

## Finding all the news headlines and  news sources

In [ ]:
news = soup.find_all("h3", class_="Mb(5px)")
print "Number of news: %d\n" % (len(news))
titles = []
sources = []
for element in news:
    t = element.a
    titles.append(t.get_text())
    s = t['href']
    sources.append(s)
    print t.get_text()
    print data_source + s
    

In [ ]:
 
with open("headlines.csv", 'wb') as file:
    writer = csv.writer(file, delimiter=',')
    for t, s in zip(titles, sources):
        writer.writerow([t.encode('utf-8').strip(), s])

## Getting world market data from Yahoo Finance

In [ ]:
#different url here
data_source = 'https://finance.yahoo.com/world-indices'
req = urllib2.Request(data_source, headers={'User-Agent': 'Mozilla/5.0'})
page = urllib2.urlopen(req).read()
soup= BeautifulSoup(page, "html5lib")
markets = soup.find("div", class_="Ovx(a)")

#print markets.prettify()        

In [ ]:
symbols = {}
rows = markets.find_all('li')

for row in rows:
    
    symbol = row.find_all('a')
    price = row.find_all('span', class_="Trsdu(0.3s) Fz(s) Mb(0px) D(b)")
    change = row.find_all('span', class_="Trsdu(0.3s) Fz(s) D(b) C($dataGreen)")
    
    if not change:
        change = row.find_all('span', class_="Trsdu(0.3s) Fz(s) D(b) C($dataRed)")
    if not (symbol):
        continue
        
    url = data_source + symbol[0].get('href')
    name = symbol[0].get('title')
    val = price[0].get_text().strip()
    chng = change[0].get_text().strip().split()
    symbols[name] = [url, val, chng[0]]
    print name, symbols[name]


In [ ]:
with open("markets.csv", 'wb') as file:
    writer = csv.writer(file, delimiter=';')
    for name, vals in symbols.items():
        line = [name]
        line.extend(vals)
        print line
        writer.writerow(line)

## Historical data from Yahoo

In [ ]:
data_source = 'https://finance.yahoo.com/quote/SNAP/history?period1=1503270736&period2=1534806736&interval=1d&filter=history&frequency=1d'
req = urllib2.Request(data_source, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'})
page = urllib2.urlopen(req).read()
soup = BeautifulSoup(page, "html5lib")

snap = soup.find("table", class_="W(100%) M(0)")

snap_header = snap.find('thead')
headers = snap_header.find_all('th')

snap_body = snap.find('tbody')
rows = snap_body.find_all('tr')

with open("snap.csv", 'wb') as file:
    writer = csv.writer(file, delimiter=',')
    header = [h.get_text().strip().replace("*", "") for h in headers]
    print header  
    writer.writerow(header)
    for row in rows:
        line = row.get_text().strip().split()
        date_tag = row.find_all("td", class_="Py(10px) Ta(start) Pend(10px)")
        if date_tag:
            values_tag = row.find_all("td", class_="Py(10px) Pstart(10px)")
            line = [date_tag[0].get_text().strip().replace(",", "")] + [v.get_text().strip().replace(",", "") for v in values_tag]
            print line
            writer.writerow(line)

